<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

This POC is intended to be a schedule generator for schools.

There are three type of resources:
 - Classrooms
 - Teachers
 - Subjects

For this POC we will consider that all students in a class go toguether and don't move, so they are something like attached to the classroom and therefore not considered as resources, but they could.

## POC

In this first POC, we are going to consider two of the three types of resources as constant, and one of them as variable. For example:
 - 1 classroom
 - 1 teacher
 - Multiple subjects (where number of subjects is greater than segment length)

This way, we just need to use resources of one type.


In [0]:
import numpy

In [0]:
# Population (total size of time slots)
POPULATION_SIZE = 100
# Each segment represents a day
SEGMENT = 5
# Five days, since the schedule is weekly based, that are 5 segments
INDIVIDUAL_SIZE = 25
# Resources, currently, of just one type, let's say subjects
RESOURCES = 9
# Number of individuals that pass to the next generation
SURVIVAL_RATE = .3
# Rate of elements that are going to mutate
MUTATION_RATE = .1
# Rate of genes that are going to be mutate
MUTATIONS = .2
# Number of iterations we do in order to get the best approach
STEPS = 500
# Positions that must remain empty
EMPTY = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
# Severity
SEVERITY = {
    'high': 6,
    'medium': 3,
    'low': 1
}

In [0]:
def create_individual(resources_size, individual_size):
  individual = numpy.random.randint(resources_size + 1, size=individual_size)
  full = [numpy.abs(index - 1) for index in EMPTY]
  individual = [item * full[index]  for index, item in enumerate(individual)]
  return individual  

In [5]:
create_individual(RESOURCES, INDIVIDUAL_SIZE)


[0, 2, 1, 0, 7, 3, 3, 8, 5, 9, 2, 9, 6, 4, 5, 0, 2, 6, 0, 4, 1, 9, 5, 0, 0]

In [0]:
def initialize_population(population_size, individual_size, resources_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([create_individual(resources_size, individual_size)])
    else:
      population = numpy.append(population, [create_individual(resources_size, individual_size)], axis=0)
  return population

In [0]:
def get_fitness(the_try):
  fitness = 0
  for i in range(int(numpy.ceil(INDIVIDUAL_SIZE / SEGMENT))):
    # print(f'Evaluating', i*SEGMENT, (i+1)*SEGMENT, the_try[i*SEGMENT:(i+1)*SEGMENT], 'from', the_try)
    unique, counts = numpy.unique(the_try[i*SEGMENT:(i+1)*SEGMENT], return_counts=True)
    # Not repeated resources in same segment
    # Except 0
    count = dict(zip(unique, counts))
    if 0 in count.keys():
      count.pop(0)
    fitness += sum(SEVERITY['medium'] for item in count.values() if item > 1)

  # Some positions must remain empty
  fitness += sum(EMPTY[index] * SEVERITY['high'] * item / item for index, item in enumerate(the_try) if item != 0)
  # Not empty slots in try unless it's mandatory
  fitness += sum(SEVERITY['medium'] for index, item in enumerate(the_try) if item == 0 and EMPTY[index] == 0) 
  return fitness

In [0]:
def get_selection(population):
  """
  First, remove duplicates
  Then leave just the SURVIVAL_RATE of the total population
  Finally sort them by fitness
  """
  selection = numpy.unique(population, axis=0)
  selection = numpy.array(sorted([[get_fitness(the_try), the_try] for the_try in selection], key=lambda x:x[0])[:int(numpy.ceil(POPULATION_SIZE * SURVIVAL_RATE))])
  return selection


In [0]:
def breed(selection):
  population = numpy.array([the_try for the_try in selection])
  i = 0
  while len(population) < POPULATION_SIZE:
    if i <= len(selection):
      parents = population[i:i+2]
      numpy.random.shuffle(parents)
      slicery = numpy.random.randint(INDIVIDUAL_SIZE) 
      population = numpy.append(population, numpy.array([numpy.append(parents[0][slicery:], parents[1][:slicery], axis=0)]), axis=0)
      i += 1
    # Once reached the end, create new random individuals
    else:
      population = numpy.append(population, [create_individual(RESOURCES, INDIVIDUAL_SIZE)], axis=0)

  return population

In [0]:
def mutate(population):
  mutated_population = population.copy()
  mutants = int(numpy.ceil(POPULATION_SIZE * MUTATION_RATE))
  for i in range(mutants):
    index = numpy.random.randint(POPULATION_SIZE)
    # Do NOT mutate accurate individuals
    if get_fitness(mutated_population[index]) > 0:
      mutant = mutated_population[index]
      num_mutations = int(numpy.ceil(INDIVIDUAL_SIZE) * MUTATIONS)
      for mutation in range(num_mutations):
        mutant[numpy.random.randint(INDIVIDUAL_SIZE)] = numpy.random.randint(1, RESOURCES)
      mutated_population[index] = mutant
  return mutated_population

In [74]:
population = initialize_population(POPULATION_SIZE, INDIVIDUAL_SIZE, RESOURCES)
for i in range(STEPS):
  print(f'Step {i}')
  selection = get_selection(population)
  print(selection[0:3])
  if all([item[0] == 0 for item in selection[0:3]]):
    break
  population = breed(selection[:,1])
  population = mutate(population)

for i in range(3):
  print(numpy.reshape(selection[:,1][i], (5, 5)))


Step 0
[[6.0
  array([2, 5, 7, 2, 6, 5, 1, 3, 8, 4, 5, 3, 5, 8, 7, 7, 2, 5, 4, 3, 5, 8,
       3, 0, 0])]
 [6.0
  array([6, 1, 7, 8, 4, 4, 9, 5, 7, 8, 0, 4, 3, 9, 7, 6, 5, 7, 1, 4, 2, 9,
       2, 0, 0])]
 [9.0
  array([3, 9, 2, 0, 7, 8, 8, 0, 1, 4, 5, 6, 3, 7, 9, 9, 2, 5, 8, 6, 5, 9,
       2, 0, 0])]]
Step 1
[[6.0
  array([2, 5, 7, 2, 6, 5, 1, 3, 8, 4, 5, 3, 5, 8, 7, 7, 2, 5, 4, 3, 5, 8,
       3, 0, 0])]
 [6.0
  array([6, 1, 7, 8, 4, 4, 9, 5, 7, 8, 0, 4, 3, 9, 7, 6, 5, 7, 1, 4, 2, 9,
       2, 0, 0])]
 [9.0
  array([3, 9, 2, 0, 7, 8, 8, 0, 1, 4, 5, 6, 3, 7, 9, 9, 2, 5, 8, 6, 5, 9,
       2, 0, 0])]]
Step 2
[[3.0
  array([5, 7, 1, 2, 3, 9, 1, 8, 2, 6, 1, 5, 7, 4, 5, 6, 2, 7, 1, 8, 3, 5,
       6, 0, 0])]
 [6.0
  array([2, 5, 7, 2, 6, 5, 1, 3, 8, 4, 5, 3, 5, 8, 7, 7, 2, 5, 4, 3, 5, 8,
       3, 0, 0])]
 [6.0
  array([4, 2, 1, 6, 3, 5, 6, 8, 1, 9, 7, 1, 7, 6, 4, 5, 7, 5, 5, 5, 2, 1,
       8, 0, 0])]]
Step 3
[[3.0
  array([5, 7, 1, 2, 3, 9, 1, 8, 2, 6, 1, 5, 7, 4, 5, 6, 2, 7, 1, 8, 3, 